# Fine-tune mobilenet v2 with SageMaker
  
This notebook should be run in a SageMaker Studio Jupyter notebook with a small-ish instance type (ml.t3.medium is fine).  
Training job will be run on a separate, more powerful instance as defined below.

### Start sagemaker session

In [2]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Define model and training instance type

In [7]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.estimator import Estimator

model_id, model_version = "tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4", "*"
training_instance_type = "ml.g4dn.xlarge"

### Retrieve training uris

In [8]:
# Retrieve the Docker image
train_image_uri = image_uris.retrieve(model_id=model_id,model_version=model_version,image_scope="training",instance_type=training_instance_type,region=None,framework=None)

# Retrieve the training script
train_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="training")

# Retrieve the pretrained model tarball for transfer learning
train_model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope="training")

### Define hyperparameters

In [9]:
# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epochs"] = "5"
hyperparameters["augmentation"] = 'True'
print(str(hyperparameters).replace(", ", "\n"))

{'train_only_top_layer': 'True'
'epochs': '5'
'batch_size': '32'
'optimizer': 'adam'
'learning_rate': '0.001'
'beta_1': '0.9'
'beta_2': '0.999'
'momentum': '0.9'
'epsilon': '1e-07'
'rho': '0.95'
'initial_accumulator_value': '0.1'
'reinitialize_top_layer': 'Auto'
'early_stopping': 'False'
'early_stopping_patience': '5'
'early_stopping_min_delta': '0.0'
'dropout_rate': '0.2'
'regularizers_l2': '0.0001'
'label_smoothing': '0.1'
'image_resize_interpolation': 'bilinear'
'augmentation': 'True'
'augmentation_random_flip': 'horizontal_and_vertical'
'augmentation_random_rotation': '0.2'
'augmentation_random_zoom': '0.1'
'binary_mode': 'False'
'eval_metric': 'accuracy'
'validation_split_ratio': '0.2'
'random_seed': '123'}


### Set up S3 input and output buckets

In [10]:
# The data is available in the following S3 bucket
training_data_bucket = "isicbucket"
training_data_prefix = "preprocessed/preprocess_ps_224"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

training_job_name = "isic-mobilenet-v2-finetune"

# Model and model artefacts will be saved to the following S3 bucket
output_bucket = sess.default_bucket()
output_prefix = "isic_mobilenet-v2_finetune"
s3_output_location = f"s3://{output_bucket}/{output_prefix}/ps_output"

In [11]:
s3_output_location

's3://sagemaker-us-west-2-766088526747/isic_mobilenet-v2_finetune/ps_output'

### Create and fit Sagemaker Estimator

In [12]:
# Create SageMaker Estimator instance
tf_ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name = training_job_name
)

# Use S3 path of the training data to launch SageMaker TrainingJob
tf_ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: isic-mobilenet-v2-finetune-2024-03-07-21-49-05-453


2024-03-07 21:49:05 Starting - Starting the training job...
2024-03-07 21:49:22 Starting - Preparing the instances for training...
2024-03-07 21:49:59 Downloading - Downloading input data......
2024-03-07 21:51:05 Downloading - Downloading the training image............
2024-03-07 21:53:01 Training - Training image download completed. Training in progress...2024-03-07 21:53:12.628773: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-03-07 21:53:12.629299: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-03-07 21:53:12.629552: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
20

## Extract Training performance metrics

In [15]:
training_job_name = tf_ic_estimator.latest_training_job.job_name
training_job_name

'isic-mobilenet-v2-finetune-2024-03-07-21-49-05-453'

In [17]:
sagemaker_session.boto_region_name

'us-west-2'

In [18]:
from sagemaker import TrainingJobAnalytics

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()

df.head(10)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


KeyError: 'MetricDefinitions'

### Deploy model for inference

In [ ]:
inference_instance_type = "ml.g4dn.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = tf_ic_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

### Download example images for inference

In [3]:
# s3_bucket = "isicbucket"
# key_prefix = "ISIC_Raw_Images/"

# def download_from_s3(images):
#     for filename, image_key in images.items():
#         boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


# test_images = {
#     "img1.jpg": "roses/10503217854_e66a804309.jpg",
#     "img2.jpg": "sunflowers/1008566138_6927679c8a.jpg",
# }
# download_from_s3(flower_images)

NameError: name 'flower_images' is not defined